In [1]:
!pip install transformers

In [2]:
import pandas as pd
import numpy as np

df_train = pd.read_csv("../input/feedback-prize-english-language-learning/train.csv")
df_train

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5


In [3]:
import tensorflow as tf
import tensorflow.keras as keras 
from transformers import BertTokenizer , TFBertModel 
bert_path = '../input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased'


tokenizer = BertTokenizer.from_pretrained(bert_path)
model = TFBertModel.from_pretrained(bert_path)

2022-10-16 12:03:47.797581: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-16 12:03:47.798752: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-16 12:03:47.799457: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-16 12:03:47.801514: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [4]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1592633663696213512
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15774515200
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9166760803154426273
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


2022-10-16 12:03:58.385209: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-16 12:03:58.385893: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-16 12:03:58.386243: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-16 12:03:58.386656: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-16 12:03:58.386998: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [5]:
def encode(input_text):
    inputs = tokenizer.batch_encode_plus(input_text,padding='max_length',max_length=512, truncation=True)
    return inputs

In [6]:
BATCH_SIZE = 16
AUTO = tf.data.experimental.AUTOTUNE

train_input = encode(df_train['full_text'].values.tolist())['input_ids']

train_data_ds = (
    tf.data.Dataset
    .from_tensor_slices((train_input,df_train.drop(['full_text', 'text_id'], axis = 1)))
    .repeat()
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

In [7]:
df_test = pd.read_csv("../input/feedback-prize-english-language-learning/test.csv")
df_test

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [8]:
testing_input = encode(df_test.full_text.values.tolist())['input_ids']

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(testing_input)
    .batch(BATCH_SIZE)
)

In [9]:
class BertLastLayerTuner(keras.Model):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.pooling = keras.layers.GlobalMaxPooling1D()
        self.last = keras.Sequential([
            keras.layers.Dense(32, activation='relu'),
            keras.layers.Dense(6)
        ])
    
    def __call__(self, batch):
        output = self.model(batch)['last_hidden_state']
        output = self.pooling(output)
        return self.last(output)
    
bert_tuner = BertLastLayerTuner(model)

In [10]:
DATASET_SIZE = df_train.shape[0] // BATCH_SIZE

train_size = int(0.85 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)

train_dataset = train_data_ds.skip(train_size)
val_dataset = train_data_ds.skip(val_size)

In [11]:
from tqdm.auto import tqdm

optimizer = tf.keras.optimizers.Adam(1e-4)
criterion = keras.losses.mean_squared_error

def compute_loss(model, x, y):
    output = model(x)
    return criterion(y_true=y, y_pred=output)
    
@tf.function
def train_step(model, x, y):
    loss = compute_loss(model, x, y)
    with tf.GradientTape() as tape:
        loss = compute_loss(model, x, y)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def train(model, train_loader, LIM_ITERATIONS = 1):
    pbar = tqdm(train_loader, total = LIM_ITERATIONS)
    sum_loss, cnt_loss = 0, 1e-5
    
    for i, batch in enumerate(pbar):
        loss = train_step(model, batch[0], batch[1])
        sum_loss += sum(loss)
        cnt_loss += 1
        pbar.set_description(f"mean_loss: {sum_loss / cnt_loss}")
        if i == LIM_ITERATIONS:
            break
    
def validate(model, valid_loader, LIM_ITERATIONS = 1):
    pbar = tqdm(valid_loader, total = LIM_ITERATIONS)
    sum_loss, cnt_loss = 0, 1e-5
    
    for i, batch in enumerate(pbar):
        loss = compute_loss(model, batch[0], batch[1])
        sum_loss += sum(loss)
        cnt_loss += 1
        pbar.set_description(f"mean_loss: {sum_loss / cnt_loss}")
        if i == LIM_ITERATIONS:
            break

In [12]:
EPOCHS = 15

for epoch in tqdm(range(EPOCHS)):
    print("==============")
    print(epoch)
    print("==============")
    train(bert_tuner, train_dataset, train_size)
    validate(bert_tuner, val_dataset, val_size)
    bert_tuner.save_weights(f'./bert_{epoch}')


  0%|          | 0/15 [00:00<?, ?it/s]

0


  0%|          | 0/207 [00:00<?, ?it/s]

2022-10-16 12:05:00.625814: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


  0%|          | 0/36 [00:00<?, ?it/s]

...

  0%|          | 0/207 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

14


  0%|          | 0/207 [00:00<?, ?it/s]

  0%|          | 0/36 [00:00<?, ?it/s]

In [13]:
df_submission = pd.read_csv("../input/feedback-prize-english-language-learning/sample_submission.csv")
df_submission

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,3.0,3.0,3.0,3.0,3.0,3.0


In [14]:
columns = ["cohesion"	,"syntax"	,"vocabulary",	"phraseology",	"grammar"	,"conventions"]

In [15]:
for i in range(len(testing_input)):
    output = bert_tuner(tf.constant([testing_input[i]])).numpy()
    for j in range(len(columns)):
        df_submission[columns[j]].iloc[i] = abs(output[0][j])

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
df_submission

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,2.850196,2.554876,3.134401,3.192795,2.637648,2.794469
1,000BAD50D026,2.497339,2.475022,2.869392,2.815385,2.355863,2.944323
2,00367BB2546B,3.408175,3.161522,3.674980,4.108593,3.432879,3.609812


In [17]:
df_submission.to_csv('submission.csv',index=False)
